# Extract data
#### 从数据标注人员那收到完整的数据，并合成大的data.csv。
- 如果没有遗漏的情况，每条data都会有三次标注。
- 如果有两条或以上的标注相同，则将该条数据用多人认可的标签写入data.csv。
- 将与多数标签不同的该条找到标注者信息然后写入wrong.csv。
- 如果三条标注的标签都不相同，把三种标签和该条item连结，写入unsure.csv
- 读入wrong.csv并进行统计。

In [3]:
#coding=utf-8
import numpy
import pandas
import csv
import numpy as np
import os
from_path="./../mid_data/checked_tag_data/4"
to_path="../../mid_data/training_data/mda_data/4"
find_file="../../mid_data/sentence_allocate"

if not os.path.exists(to_path):
    os.makedirs(to_path)

In [2]:
'''
check if check_item has been in path.
input: 
    check_item: A line from csv
    path: An csv file waiting to check
output: True / False
''' 
def Ifexists(check_item,path):
    #return False
    file_list=[]
    file=csv.reader(open(path,'r',encoding='utf-8-sig'))
    for item in file:
        file_list.append(item)
    if check_item in file_list:
        return True
    return False


'''
find the one submit the file 
input: 
    file_to_find
output:
    the string who submit the file
    or if no one submit it, return False
'''
def FindFilename(file_to_find):    
    for parent,dirnames,filenames in os.walk(find_file):         
        for filename in filenames:
            if filename==file_to_find:                
                return parent
    return False

'''#identify if the tag is legel'''
def IfLeagel(tag):
    if tag=='' or (tag!='0' and tag!='1' and tag!='-1' and tag!='-2'):
        return False
    return True

In [4]:
'''sort out the file names of csvs into a list'''
csv_list=[]
for parent,dirnames,filenames in os.walk(from_path): 
    for filename in filenames:
        csv_list.append(filename)
csv_list.sort()


'''traversing'''
p=0
while(p<len(csv_list)):

    filename=csv_list[p]
    csv_group=[]
    
    '''make a group of every three csvs with the same content''' 
    for csv_item in csv_list:
        if csv_item[:-6]==filename[:-6]:
            csv_group.append(csv_item)
    
    
    p += len(csv_group)
    if len(csv_group)==1:
        continue
        
    '''if the group only has two files submitted,for every item, 
        record it if this two files have the same tag.
    '''    
    elif len(csv_group)==2:    
        file1=csv.reader(open(os.path.join(from_path,csv_group[0]),'r',encoding='utf-8-sig')) 
        file2=csv.reader(open(os.path.join(from_path,csv_group[1]),'r',encoding='utf-8-sig'))
        items1=[]
        items2=[]
        
        print(csv_group[0])
        for item in file1:            
            '''solve the problem that some tagged the last place rather than the forth blank'''
            if item[4]=='' and len(item[-1])<=2:
                item[4]=item[-1]
                del item[-1]
            '''solve the problem that some tagged "2" wrong as "-2" '''
            if item[4]=='2':
                item[4]='-2'
            items1.append(item)
            
        print(csv_group[1])       
        for item in file2:
            '''solve the problem that some tagged the last place rather than the forth blank'''
            if item[4]=='' and len(item[-1])<=2:
                item[4]=item[-1]
                del item[-1]  
            '''solve the problem that some tagged "2" wrong as "-2" ''' 
            if item[4]=='2':
                item[4]='-2'
            items2.append(item)
                  
        '''fatal error'''
        assert len(items1)==180
        assert len(items2)==180
        
        items_list=[items1,items2]
        
        for i in range(len(items1)):           
            '''Verify the label legitimacy of each record'''
            fwrong=open(os.path.join(to_path,"wrong.csv"),'a+',encoding='utf-8-sig',newline="\n")
            for index,items in enumerate(items_list):                 
                if not IfLeagel(items[i][4]):
                    statistic_struct=items[i].copy()
                    statistic_struct.append(FindFilename(csv_group[index]).split('\\')[1])                   
                    statistic_struct.append(str(csv_group[index]))
                    
                    if not Ifexists(statistic_struct,"wrong.csv"):
                        fwrong_write = csv.writer(fwrong,dialect='excel')
                        fwrong_write.writerow(statistic_struct) 
                    else:
                        print("the item%s has already been recorded" %(str(statistic_struct)))
            fwrong.close()
            
            '''compare the two labels，if they are the same,write in data.csv, else, continue'''
            ftrue=open(os.path.join(to_path,"data.csv"),'a+',encoding='utf-8-sig',newline="\n")
            
            if items1[i]==items2[i]:
                if not Ifexists(items1[i],os.path.join(to_path,"data.csv")):
                    ftrue_write=csv.writer(ftrue,dialect='excel')
                    ftrue_write.writerow(items1[i])
#             else:
#                 print("items1[i]:",items1[i])
#                 print("items2[i]:",items2[i])
            ftrue.close() 
            
          
    elif len(csv_group)==3:
        file1=csv.reader(open(os.path.join(from_path,csv_group[0]),'r',encoding='utf-8-sig')) 
        file2=csv.reader(open(os.path.join(from_path,csv_group[1]),'r',encoding='utf-8-sig'))
        file3=csv.reader(open(os.path.join(from_path,csv_group[2]),'r',encoding='utf-8-sig')) 
        
        items1=[]
        items2=[]
        items3=[]
        
        print(csv_group[0])
        for item in file1:
            if item[4]=='' and len(item[-1])<=2:
                item[4]=item[-1]
                del item[-1]
            if item[4]=='2':
                item[4]='-2'   
            items1.append(item)
            
        print(csv_group[1])
        for item in file2:
            if item[4]=='' and len(item[-1])<=2:
                item[4]=item[-1]
                del item[-1]
            if item[4]=='2':
                item[4]='-2'               
            items2.append(item) 
            
        print(csv_group[2])
        for item in file3:
            if item[4]=='' and len(item[-1])<=2:
                item[4]=item[-1]
                del item[-1]
            if item[4]=='2':
                item[4]='-2'                
            items3.append(item)  
            
        
        assert len(items1)==180
        assert len(items2)==180  
        assert len(items3)==180
        
        
        items_list=[items1,items2,items3]
        
        for i in range(len(items1)):
            
            '''Verify the label legitimacy of each record'''
            fwrong=open(os.path.join(to_path,"wrong.csv"),'a+',encoding='utf-8-sig',newline="\n")
            for index,items in enumerate(items_list):                 
                if not IfLeagel(items[i][4]):
                    statistic_struct=items[i].copy()
                    statistic_struct.append(FindFilename(csv_group[index]).split('\\')[1])                    
                    statistic_struct.append(str(csv_group[index]))
                    
                    if not Ifexists(statistic_struct,os.path.join(to_path,"wrong.csv")):
                        fwrong_write = csv.writer(fwrong,dialect='excel')
                        fwrong_write.writerow(statistic_struct) 
                    else:
                        print("the item%s has already been recorded" %(str(statistic_struct)))
            
            funsure=open(os.path.join(to_path,"unsure.csv"),'a+',encoding='utf-8-sig',newline="\n")
            ftrue=open(os.path.join(to_path,"data.csv"),'a+',encoding='utf-8-sig',newline="\n")
            
            '''if the three items are the same, adding additional remarking and write it into data.csv'''
            if items1[i]==items2[i] and items2[i]==items3[i]:
                if not Ifexists(items1[i],os.path.join(to_path,"wrong.csv")):
                    statistic_struct=items1[i].copy()                   
                    statistic_struct.append("1")
                    ftrue_write=csv.writer(ftrue,dialect='excel')
                    ftrue_write.writerow(statistic_struct)
                    
            '''if through three items, two items are the same, confirm the label, write it into data.csv.
            For the other one, find the one who handed in the file and record it in wrong.csv'''
            elif items1[i] == items2[i] and items2[i] != items3[i]:
                if not Ifexists(items1[i],os.path.join(to_path,"data.csv")):
                    ftrue_write=csv.writer(ftrue,dialect='excel')
                    ftrue_write.writerow(items1[i])
                statistic_struct=items3[i].copy()
                statistic_struct.append(FindFilename(csv_group[2]).split('\\')[1])
                statistic_struct.append(str(csv_group[2]))
                statistic_struct[4]=statistic_struct[4]+"->"+items1[i][4]
                if IfLeagel(items3[i][4]) and items2[i][4]!=items3[i][4] and not Ifexists(statistic_struct,os.path.join(to_path,"wrong.csv")):                    
                    fwrong_write = csv.writer(fwrong,dialect='excel')
                    fwrong_write.writerow(statistic_struct) 
                else:
                    print("the item%s has already been recorded" %(str(statistic_struct)))               
            elif items1[i] == items3[i] and items2[i] != items1[i]:
                if not Ifexists(items1[i],os.path.join(to_path,"data.csv")):
                    ftrue_write=csv.writer(ftrue,dialect='excel')
                    ftrue_write.writerow(items1[i])
                statistic_struct=items2[i].copy()
                
                statistic_struct.append(FindFilename(csv_group[1]).split('\\')[1])
                statistic_struct.append(str(csv_group[1]))
                statistic_struct[4]=statistic_struct[4]+"->"+items1[i][4]
                if IfLeagel(items2[i][4]) and items2[i][4]!=items1[i][4] and not Ifexists(statistic_struct,os.path.join(to_path,"wrong.csv")):
                    
                    fwrong_write = csv.writer(fwrong,dialect='excel')
                    fwrong_write.writerow(statistic_struct) 
                else:
                    print("the item%s has already been recorded" %(str(statistic_struct)))
            elif items2[i] == items3[i] and items1[i] != items2[i]:
                if not Ifexists(items1[i],os.path.join(to_path,"data.csv")):
                    ftrue_write=csv.writer(ftrue,dialect='excel')
                    ftrue_write.writerow(items2[i])
                statistic_struct=items1[i].copy()
                statistic_struct.append(FindFilename(csv_group[0]).split('\\')[1])
                statistic_struct.append(str(csv_group[0]))
                statistic_struct[4]=statistic_struct[4]+"->"+items2[i][4]
                if IfLeagel(items1[i][4]) and items1[i][4]!=items2[i][4] and not Ifexists(statistic_struct,os.path.join(to_path,"wrong.csv")):
                   
                    fwrong_write = csv.writer(fwrong,dialect='excel')
                    fwrong_write.writerow(statistic_struct) 
                else:
                    print("the item%s has already been recorded" %(str(statistic_struct)))
                    
            '''If the three items give totally different label,
            then record it with three controversial labels into unnsure.csv   '''     
            else:
                statistic_struct=items1[i].copy()
                statistic_struct[4]=statistic_struct[4]+"\\"+items2[i][4]+"\\"+items3[i][4]
                if not Ifexists(statistic_struct,os.path.join(to_path,"unsure.csv")):
                    funsure_write=csv.writer(funsure,dialect='excel')
                    funsure_write.writerow(statistic_struct)
                    
            ftrue.close()
            fwrong.close()
            funsure.close()

sentence1012_0.csv
sentence1012_1.csv
sentence1012_2.csv
the item['600011', '2017', '330', '1', '->0', '公司主要在中国全国范围内开发、建设和经营管理大型发电厂。', 'C4', 'sentence1012_2.csv'] has already been recorded
the item['428', '2011', '2414', '497', '0->0', '华天实业控股集团有限公司原系军队企业，1998 年 11 月 20 日经中共广州军区委员会（1998）广党字第 15 号文件批复，于 1998 年 11 月 28 日将该公司移交湖南省人民政府，根据湘政发[2004]16 号?湖南省人民政府关于省国资委履行出资人职责有关问题的通知?，由湖南省国资委履行其出资人职责。', 'B4', 'sentence1012_1.csv'] has already been recorded
sentence1032_0.csv
sentence1032_1.csv
sentence1032_2.csv
sentence1080_0.csv
sentence1080_1.csv
sentence1080_2.csv
sentence1085_0.csv
sentence1085_1.csv
sentence1085_2.csv
sentence111_0.csv
sentence111_1.csv
sentence111_2.csv
sentence1120_0.csv
sentence1120_1.csv
sentence1120_2.csv
the item['600337', '2014', '807', '87', '11->1', '公司在“五?五规划”中明确了公司的发展战略：致力于“建世界优质企业，创全球著名品牌”，着眼于全球的优势资源，转变企业发展方式，借助资本运营平台，紧密围绕品牌战略，优化产业结构，强化高效协同的产业链一体化体系，从内涵和外延的两个角度促进跨越式增长。', 'B4', 'sentence1120_2.csv'] has already been recorded
sentence1223_0.csv
sentence1223_1.cs

the item['600606', '2014', '9239', '11', '->0', '报告期内，“金丰一号”、“金丰三号”房地产流通服务信托基金到期清算，“金丰二号”上海罗店保障房投资集合资金信托计划提前终止清算，公司金融服务业务实现管理费收入 289.78 万元。', 'G4', 'sentence333_0.csv'] has already been recorded
the item['300210', '2011', '6703', '96', '->0', '经营活动产生的现金流量净额和每股经营活动产生的现金流量均比上年减少，主要是随着公司订单增多，生产销售规模扩大，为了能及时满足市场对公司产品的需求，公司增加了原材料采购所致。', 'G4', 'sentence333_0.csv'] has already been recorded
the item['600322', '2013', '8552', '30', '->0', '土 地 整 理 区域 3300 亩，可 出 让 面 积1711 亩。', 'G4', 'sentence333_0.csv'] has already been recorded
the item['600343', '2015', '8571', '31', '->0', '如何紧紧抓住政府大力推进供给侧改革和企业节能增收、降本增效形成新的市场需求，进行产业结构调整和转型升级，破解发展瓶颈问题，是公司当前和“十三五”期间工作重心。', 'G4', 'sentence333_0.csv'] has already been recorded
the item['300158', '2012', '6410', '107', '->0', '根据公司2013年第二届董事会第十六次会议决议，公司拟以2012年末股本288,000,000股为基数，向全体股东每10股派发现金股利人民币0.3元（含税），合计派发现金864万元，剩余合并报表未分配利润结转下一年度。', 'G4', 'sentence333_0.csv'] has already been recorded
the item['861', '2011', '3601', '179', '->0', '报告期内实现营业收入 0 万元，营业利润 5.63 万元，

the item['600848', '2011', '9716', '179', '->0', '我们审计了后附的上海自动化仪表股份有限公司(以下简称“贵公司”)的财务报表，包括 2011年 12 月 31 日的公司及合并资产负债表、2011 年度的公司及合并利润表、公司及合并现金流量表和公司及合并股东权益变动表以及财务报表附注。', 'G4', 'sentence333_0.csv'] has already been recorded
the item['409', '2012', '7578', '21', '->1', '财务实行收支两条线和资金池管理，人力资源推行新的定岗、定编和薪酬管理办法，设备集中采购，产品统一销售，加强信息化建设，提高集团管控能力。', 'G4', 'sentence333_0.csv'] has already been recorded
the item['9', '2011', '2022', '124', '->0', '（2）关于授权董事局出售上海三家公司股权的议案。', 'G4', 'sentence333_0.csv'] has already been recorded
the item['600302', '2011', '8498', '302', '->0', '①可抵扣暂时性差异是在以下交易中产生的：该交易不是企业合并，并且交易发生时既不影响会计利润也不影响应纳税所得额或可抵扣亏损。', 'G4', 'sentence333_0.csv'] has already been recorded
the item['600755', '2015', '1462', '81', '->1', '公司将借鉴和推广不同地区的管理文化和成功经验，有效利用各区域平台扩大规模化经营。', 'G4', 'sentence333_0.csv'] has already been recorded
the item['300246', '2012', '6919', '18', '->1', '报告期内，公司持续加大对研发和营销投入并采取稳健的经营策略，成功的收购了挚信鸿达60%的股权，这在一定程度上维系了公司良好的财务状况、实现了公司业绩的平稳增长、为公司业绩长期可持续增长奠定了基础。', 'G4', 'sentence333_

the item['538', '2017', '2598', '92', '1->1', '2018年，我们将继续围绕“新白药，大健康”战略，基于即有的资金优势、品牌优势、渠道优势和完全市场化的体制优势，巩固和提升现有业务板块，由点及面地构建从金融端?资本端?产业端?产品端?服务端的完整产业生态圈，以培育云南白药全新的创新能力。', 'B4', 'sentence3610_0.csv'] has already been recorded
sentence3774_0.csv
sentence3774_1.csv
sentence3774_2.csv
sentence444_0.csv
sentence444_1.csv
sentence444_2.csv
the item['300043', '2017', '5793', '66', '0->0', 'Sport的数据报告，截止2017年12月底，皇家西班Wong）加入西班牙?二队，代表球队参加多项国内牙?俱乐部球员总价值已达1.05亿欧元，较上赛季外赛事，并在同印尼U19国家队比赛中打入?球。', 'B4', 'sentence444_2.csv'] has already been recorded
sentence457_0.csv
sentence457_1.csv
sentence457_2.csv
sentence511_0.csv
sentence511_1.csv
sentence511_2.csv
sentence636_0.csv
sentence636_1.csv
sentence636_2.csv
the item['600739', '2013', '1432', '8', '-->0', '在生产上，继续坚持把产品质量作为企业生存的根本，不断提升质量管理水平。', 'B4', 'sentence636_1.csv'] has already been recorded
sentence645_0.csv
sentence645_1.csv
sentence645_2.csv
sentence704_0.csv
sentence704_1.csv
sentence704_2.csv
sentence715_0.csv
sentence715_1.csv
sentence715_2.cs

# wrong.csv 文件的信息抽取

In [5]:

wrong_dict={"A4":0,"B4":0,"C4":0,"D4":0,"E4":0,"F4":0,"G4":0,"H4":0}
wrong_tagged_dict=wrong_dict.copy()
tagged_sum={"A4":30*180,"B4":30*180,"C4":30*180,"D4":30*180,"E4":30*180,"F4":30*180,"G4":30*180,"H4":30*180}

for line in csv.reader(open(os.path.join(to_path,"wrong.csv"),mode='r',encoding='utf-8-sig')):
    if line[6] in list(wrong_dict.keys()):
        wrong_dict[line[6]]+=1
    
        if line[4].find("->")==-1:
            wrong_tagged_dict[line[6]]+=1
print("----------一共错的--------------")   
print(wrong_dict)       
print("----------手滑错的--------------")   
print(wrong_tagged_dict)
print("----------错误率--------------")   
for (key,value) in wrong_dict.items():
    print(str(key)+":"+str(value/tagged_sum[key]*1.0))

----------一共错的--------------
{'H4': 1091, 'F4': 467, 'B4': 554, 'C4': 951, 'D4': 918, 'G4': 662, 'A4': 562, 'E4': 958}
----------手滑错的--------------
{'F4': 2, 'B4': 8, 'C4': 2, 'D4': 3, 'H4': 0, 'E4': 1, 'G4': 149, 'A4': 0}
----------错误率--------------
H4:0.20203703703703704
F4:0.08648148148148148
B4:0.1025925925925926
C4:0.1761111111111111
D4:0.17
G4:0.12259259259259259
A4:0.10407407407407407
E4:0.1774074074074074
